# **Laboratorio para pruebas**

### **Librerias Necesarias**

In [2]:
%run Proyecto_idDrRecursos.ipynb

In [3]:
Freqs_drone = pd.read_json("Seleccion/colFreqs6000.json")
label_drone = pd.read_csv("Seleccion/labelFreqs6000.csv")
#Configuraciones para no truncar la salida  ***************
pd.set_option('display.max_rows', None)
#Porcentaje 50 
percent_query = 30
k = 3

In [4]:
#Datos 

#Query
nume_samples_query = int((len(Freqs_drone) * percent_query)/100)                       #Numero de muestras con respecto al porcentanje         
query_index_samples = random.sample(list(range(0,400)),nume_samples_query)          #Escogemos los indices de query aleatoriamente sin remplazo
query_index_samples.sort()                                                          #Ordenamos 
query_samples = Freqs_drone.iloc[query_index_samples]                                  #seleccionamos de data_num con respecto a los indices encontrados 
#Training
#Agregar inplace como True para eliminar de la original 
training_samples = Freqs_drone.drop(query_index_samples)                               #Escogemos los indices faltantes

#Label 
query_samples_label = label_drone.iloc[query_index_samples]                          #iloc -> [] usando los indices antes encontrados para query
training_samples_label = label_drone.drop(query_index_samples)                       #Escogemos los indices faltantes 

#Elementos en total 
total = len(query_samples)

In [6]:

cols_dn = len(Freqs_drone.columns)                                                     #Columnas de data_num
size_ts = len(training_samples)                                                     #Size de training_samples
size_q  = len(query_samples)                                                        #Size de query
k_DI = []                                                                           #Aqui se almacenan las k distancias mas pequeñas                                                                          #
for j in range(size_q):                                                             #iteramos por todos los elementos de la query
    Q = np.tile(query_samples.iloc[j,:],(training_samples.shape[0],1))              #Igual que 'repmat'
    Z = Q - training_samples                                                        #
    
    S_with_garbage = Z @ Z.T                                                        #puede usar tambien np.dot(m1,m2)
    distancia_S = np.diag(S_with_garbage) ** (1 / 2)                                #Obtener la diagonal y elevar sus elem a 1/2
    
    sort_index = np.argsort(distancia_S)                                            #Obtener los indices ordenados
    
    k_index = sort_index[:k]                                                        #Seleccionamos los k primeros
    k_dis = distancia_S[k_index]                                                    #Almacenamos las k distancias minimas

    mode_k_dis = mode(k_dis)[0]                                                     #mode retorna (moda, repeticiones)
    index_mode = np.where(distancia_S == mode_k_dis)[0][0]                          #buscamos indice en distancia_S de mode_k_dis
    """
    index_mode1 = 0                                                                 #La primera forma que se me ocurrio de hacerlo
    for i in k_index:                                                               #luego encontre la solución de arriba
        if distancia_S[i] == mode_k_dis:
            index_mode = i
            break
    print(distancia_S[index_mode],distancia_S[index_mode1])
    """
    k_DI.append(index_mode) 

cont_Error = 0
for i in range(len(query_samples_label)):
    print(f"Label en query:{query_samples_label.iloc[i]}")
    print(f"Valor dado por kNN:{training_samples_label.iloc[k_DI[i]]}\n")
    if str(query_samples_label.iloc[i][0]) != str(training_samples_label.iloc[k_DI[i]][0]):
        cont_Error += 1
        print("------",str(query_samples_label.iloc[i][0]))
        print(f"Label {i} en query:{query_samples_label.iloc[i]}")
        print(f"Valor dado por kNN:{training_samples_label.iloc[k_DI[i]]}\n")

Label en query:0    Drone
Name: 0, dtype: object
Valor dado por kNN:0    Drone
Name: 181, dtype: object

Label en query:0    Drone
Name: 4, dtype: object
Valor dado por kNN:0    Drone
Name: 7, dtype: object

Label en query:0    Drone
Name: 5, dtype: object
Valor dado por kNN:0    Drone
Name: 99, dtype: object

Label en query:0    Drone
Name: 9, dtype: object
Valor dado por kNN:0    Drone
Name: 190, dtype: object

Label en query:0    Drone
Name: 12, dtype: object
Valor dado por kNN:0    Drone
Name: 140, dtype: object

Label en query:0    Drone
Name: 14, dtype: object
Valor dado por kNN:0    Drone
Name: 137, dtype: object

Label en query:0    Drone
Name: 15, dtype: object
Valor dado por kNN:0    Drone
Name: 156, dtype: object

Label en query:0    Drone
Name: 16, dtype: object
Valor dado por kNN:0    Drone
Name: 70, dtype: object

Label en query:0    Drone
Name: 18, dtype: object
Valor dado por kNN:0    Drone
Name: 99, dtype: object

Label en query:0    Drone
Name: 19, dtype: object
Valor

/tmp/ipykernel_179434/2936247945.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if str(query_samples_label.iloc[i][0]) != str(training_samples_label.iloc[k_DI[i]][0]):
/tmp/ipykernel_179434/2936247945.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("------",str(query_samples_label.iloc[i][0]))


In [8]:
print(f"Errores: {cont_Error}")
percent_success = (total - cont_Error) * (100 / total)
print(f"Porcentaje query seleccionado: {percent_query}\n",
      f"Porcentaje training seleccionado: {100 - percent_query}\n",
      f"Elementos en query: {len(query_samples)}\n",
      f"Elementos en training: {len(training_samples)}\n",
      f"k : {k}\n",
      f"Porcentaje exito: {percent_success}\n",
      f"Total: {total-cont_Error}", sep="")

Errores: 1
Porcentaje query seleccionado: 30
Porcentaje training seleccionado: 70
Elementos en query: 120
Elementos en training: 280
k : 3
Porcentaje exito: 99.16666666666667
Total: 119
